In [1]:
import os

In [2]:
%pwd

'c:\\Madhu\\2025_NLPDataScientist\\UKJobs\\Hiscox\\insuranceClaim\\End-to-end-MLOps-AWS-CICD-Pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Madhu\\2025_NLPDataScientist\\UKJobs\\Hiscox\\insuranceClaim\\End-to-end-MLOps-AWS-CICD-Pipeline'

In [21]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    X_train_data_path: Path
    X_test_data_path: Path
    y_train_data_path: Path
    y_test_data_path: Path
    model_name: str   
    min_child_weight: int
    max_depth: int
    target_column: str

In [22]:
from mlopsProject.constants import *
from mlopsProject.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            X_train_data_path = config.X_train_data_path,
            X_test_data_path = config.X_test_data_path,
            y_train_data_path = config.y_train_data_path,
            y_test_data_path = config.y_test_data_path,
            model_name = config.model_name,
            min_child_weight = params.min_child_weight,
            max_depth = params.max_depth,
            target_column = schema.name
            
        )

        return model_trainer_config


In [24]:
import pandas as pd
import os
from mlopsProject import logger
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

import joblib

In [33]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    
    def train(self):
        train_x = pd.read_csv(self.config.X_train_data_path)
        test_x = pd.read_csv(self.config.X_test_data_path)
        train_y = pd.read_csv(self.config.y_train_data_path)
        test_y = pd.read_csv(self.config.y_test_data_path)

        best_model = XGBClassifier(min_child_weight=self.config.min_child_weight, max_depth=self.config.max_depth, n_jobs=-1)
        best_model = best_model.fit(train_x, train_y)

        joblib.dump(best_model, os.path.join(self.config.root_dir, self.config.model_name))

In [34]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-07-17 05:56:19,159: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-17 05:56:19,161: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-17 05:56:19,170: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-17 05:56:19,172: INFO: common: created directory at: artifacts]
[2024-07-17 05:56:19,176: INFO: common: created directory at: artifacts/model_trainer]
